In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


# Configure an LLM
llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0.5,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    streaming=True,
    extra_body={"enable_thinking": False},
)

In [4]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


print(amultiply.name)
print(amultiply.description)
print(amultiply.args)

amultiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [6]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

6
10


In [5]:
tools = [calculator]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

response = llm_with_tools.invoke(query)

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_17b9218008784cd4827512', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen3-32b'}, id='run--d02cf5b5-cdf7-4923-8977-28ebc49f4b42-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_17b9218008784cd4827512', 'type': 'tool_call'}])

In [6]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_17b9218008784cd4827512',
  'type': 'tool_call'}]

In [7]:
messages = []

for tool_call in response.tool_calls:
    selected_tool = {"multiply": calculator, "amultiply": calculator}[
        tool_call["name"].lower()
    ]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[ToolMessage(content='36', name='multiply', tool_call_id='call_17b9218008784cd4827512')]

In [8]:
llm_with_tools.invoke(messages)

BadRequestError: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: messages with role "tool" must be a response to a preceeding message with "tool_calls".', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-4c5c66da-81a5-9a9a-804b-f90e2a313a5e', 'request_id': '4c5c66da-81a5-9a9a-804b-f90e2a313a5e'}

In [9]:
llm_with_tools.invoke([response] + messages)

AIMessage(content='The result of multiplying 3 and 12 is 36.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-32b'}, id='run--2d807959-7807-43b5-ad29-7d2302ad7560-0')